In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
#import libraries
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os

# Importing Deep Learning Libraries
import tensorflow as tf
from tensorflow import keras
import cv2
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D

In [2]:
# importing data
train_dir = "/kaggle/input/fer2013/train/"
test_dir = "/kaggle/input/fer2013/test/"

# **Data Exploration**

In [3]:
# function to plot top 5 images
def plot_images(img_dir, top=5):
    all_img_dirs = os.listdir(img_dir)
    img_files = [os.path.join(img_dir, file) for file in all_img_dirs][:5]
  
    plt.figure(figsize=(10, 10))
  
    for idx, img_path in enumerate(img_files):
        plt.subplot(5, 5, idx+1)
    
        img = plt.imread(img_path)
        plt.tight_layout()         
        plt.imshow(img, cmap='Blues_r')

In [4]:
plot_images(train_dir+"happy")

In [5]:
# assinging a image from train
img_array = cv2.imread(train_dir+"angry/Training_3908.jpg")

In [6]:
plt.imshow(img_array)

In [7]:
# checking image shape
img_array.shape

# **Data Preprocessing**

In [8]:
# Train data generator
datagen = ImageDataGenerator(rescale=1./255)

train_set = datagen.flow_from_directory(train_dir,
                                              target_size = (48,48),
                                              color_mode = "grayscale",
                                              batch_size=32,
                                              class_mode='categorical',
                                              shuffle=True)
train_set.class_indices

In [9]:
# Validation data generator
val_datagen = ImageDataGenerator(rescale=1./255)

validation_set = val_datagen.flow_from_directory(test_dir,
                                              target_size = (48,48),
                                              color_mode = "grayscale",
                                              batch_size=32,
                                              class_mode='categorical',
                                              shuffle=False)

validation_set.class_indices

# **Building CNN model**

In [10]:
# create model structure
model = Sequential()

#1st CNN layer
model.add(Conv2D(64,(3,3),padding = 'same',input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#2nd CNN layer
model.add(Conv2D(128,(5,5),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#3rd CNN layer
model.add(Conv2D(512,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#4th CNN layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

#Fully connected 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))



opt = Adam(lr = 0.001)
model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [12]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("./model_weights.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True)

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_lr=0.0001)

callbacks_list = [early_stopping,checkpoint,reduce_learningrate]

In [13]:
epochs = 30
steps_per_epoch= train_set.n//train_set.batch_size
validation_steps = validation_set.n//validation_set.batch_size

In [14]:
# Train the neural network/model
history = model.fit_generator(generator=train_set,
                                steps_per_epoch=steps_per_epoch,
                                epochs=epochs,
                                validation_data = validation_set,
                                validation_steps = validation_steps,
                                callbacks=callbacks_list
                                )

In [15]:
# saving model
model.save("./final_model.h5")
model.save_weights("./final_model_weights.h5")

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
# Loading the best fit model 
from keras.models import load_model
model = load_model("./final_model.h5")

# **Model Evaluation**

In [16]:
h =  history.history
h.keys()

In [17]:
# ploting accuracy
plt.plot(h['accuracy'], label='Training Accuracy')
plt.plot(h['val_accuracy'] , c = "red", label='Validation Accuracy')
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(loc='lower right')
plt.savefig("./final_cnn_accuracy.png")
plt.show()

In [18]:
# ploting loss
plt.plot(h['loss'], label='Training Loss')
plt.plot(h['val_loss'] , c = "red", label='Validation Loss')
plt.title("Training loss vs Validation Loss")
plt.legend(loc='upper right')
plt.savefig("./final_cnn_loss.png")
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

In [ ]:
def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')# **Live Class Monitoring System(Face Emotion Recognition)**
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
# Predicting For Test Set
predictions = model.predict(validation_set)

# Printing Confution Matrix
cm = confusion_matrix(y_pred=np.argmax(predictions, axis=-1), y_true=validation_set.classes)                         
cm

In [ ]:
# compute predictions
predictions = model.predict_generator(generator=validation_set)
y_pred = [np.argmax(probas) for probas in predictions]
y_test = validation_set.classes
class_names = validation_set.class_indices.keys()

# compute confusion matrix
con_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# plot confusion matrix
plt.figure()
plot_confusion_matrix(con_matrix, classes=class_names, title='Confusion matrix')
plt.savefig("./final_cnn_cm.png")
plt.show()